In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 

/Users/ankitanand/Documents/Learnings/LangGraph/Langgraph-end-to-end/Building_ReAct_from_scratch/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm.invoke("Can cars fly?").content

'As of now, traditional cars do not have the capability to fly. However, there are several prototypes and concepts for flying cars or vertical takeoff and landing (VTOL) vehicles that have been developed by various companies and inventors. These vehicles typically combine elements of both aviation and automotive design, allowing them to operate on roads and in the air.\n\nSome of these flying car prototypes have undergone testing, and a few have even received regulatory approval for limited use. However, widespread commercial availability and integration into existing transportation systems are still in the early stages and face numerous technical, regulatory, and safety challenges.\n\nIn summary, while flying cars are a concept that is being explored, they are not yet a common reality.'

In [13]:
class Chatbot():
    def __init__(self, system=""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role":"system", "content":system})

    def __call__(self,message):
        self.message.append({"role":"user", "content":message})
        result=self.execute()
        self.message.append({"role":"assistant", "content":result})
        return result

    def execute(self):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        result = llm.invoke(self.message)
        return result.content

In [14]:
bot=Chatbot(system="you are helpful assistant")

In [15]:
bot("Hi how are you")

"I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"

In [16]:
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'Hi how are you'},
 {'role': 'assistant',
  'content': "I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"}]

In [17]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [18]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [19]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [20]:
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [21]:
def calculate(number):
    return eval(number)

In [22]:
calculate("2+2")

4

In [23]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [24]:
def query(question, max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [25]:
bot = Chatbot(prompt)

In [26]:
bot("hi how are you?")

'hi, i am good.'

In [27]:
bot("What is your job?")

'Thought: I should explain my purpose and function. \nAction: wikipedia: AI assistant\nPAUSE'

In [28]:
next_prompt="Tell me about quantum computing from wikipedia"

In [33]:
next_prompt="Tell me about the current gdp of India from wikipedia"

In [34]:
result = bot(next_prompt)

In [35]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [36]:
actions

[<re.Match object; span=(0, 31), match='Action: wikipedia: GDP of India'>]

In [37]:
action, action_input = actions[0].groups()

In [38]:
action

'wikipedia'

In [39]:
action_input

'GDP of India'